In [1]:
#importing all the tools that will be neccessary

from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException


In [142]:
# we have all the links for reviews in a CSV lets retrive that
Card_info = pd.read_csv("card_name_URL.csv")

In [144]:
#now lets get those links from the dataframe into a list
card_links = Card_info["Reviews URL"].tolist()

In [131]:
# tell python where chrome driver is wrt your current working directory
path = "chromedriver"
browser = webdriver.Chrome(path)

In [146]:
#setting some empty lists to store the data in the mean time
data = []
authors = []
ratings = []
dates = []
review_texts = []
review_titles = []

In [147]:
#now lets gather all the data from each movie review link
browser = webdriver.Chrome(path)
for card in card_links:
    pagelink = card
    browser.get(pagelink)
    time.sleep(2)
    page_num = 5

    for p in range(0, page_num):
        print ('page',p+1)

        # get all the review divs
        page_source = browser.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        reviews = soup.findAll('article', {'class':re.compile('pv3')})

        # grab the information for each review
        for review in reviews:

            # initialize author, rating, date, review text, review title
            author,rating,date,review_title,review_text ='NA','NA','NA','NA','NA'

            #if there is critic name information, get it
            author_ = review.find('div',{'class':re.compile('f5 lh-copy ck-black-60 pa0 mb3')})
            if author_: 
                author = author_.text.strip()

            # if there is rating information, get it
            rating_ = review.findAll('svg',{'class':re.compile('fill-currentColor dib ck-primary-50')})
            for i in range(0,len(rating_)):
                rating = i + 1


            # if there is date information, get it  
            date_= review.find('span',{'class':re.compile("dib f5 lh-copy")})
            if date_: 
                date = date_.text.strip()

            # if there is review title information, get it 
            reviewtitles = review.find('h5',{'class':re.compile('f4 lh-title ck-black-90 mb2 mt3')})
            if reviewtitles: 
                review_title = reviewtitles.text.strip()
    
            # if there is review text information, get it 
            reviewtexts = review.find('p',{'class':re.compile('f4 lh-copy ma0')})
            if reviewtexts: 
                review_text = reviewtexts.text
            
            #lets add the data to a list
            authors.append(author)
            ratings.append(rating)
            dates.append(date)
            review_titles.append(review_title)
            review_texts.append(review_text)
        
        #moving to next page and setting a parameter incase there is no next page
        try:
            if p < page_num:
                browser.find_element_by_xpath('//div[@data-testid="reviews__pagination-next-page"]').click()
                time.sleep(2)
        except:
            continue
browser.close()

page 1


<ipython-input-147-e6dd51d294b0>:59: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath('//div[@data-testid="reviews__pagination-next-page"]').click()


page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4
page 5
page 1
page 2
page 3
page 4

In [148]:
#adding the data to a data frame
data = {
    'Authors': authors,
    'Ratings': ratings,
    'Date' : dates,
    'review_titles': review_titles,
    'review_texts': review_texts
}

df = pd.DataFrame(data)

# check the shape of our dataframe
df.shape
# take a glimpse 
df.tail()

,Authors,Ratings,Date,review_titles,review_texts
3619,Credit Karma Member,5,"Apr 16, 2017","Great Card, fits my needs","Being 4yrs post bankruptcy, I was surprised to..."
3620,Credit Karma Member,1,"Jan 27, 2017",Not worth it !!,I apply for this credit card since my credit s...
3621,Credit Karma Member,5,"Jan 07, 2017",Best Cash Back in Every Day Spending,This card loses a lot of people's interest bec...
3622,Credit Karma Member,2,"Dec 19, 2016",Barclays is cheap on credit lines,Every time Barclays gets a credit line request...
3623,Credit Karma Member,5,"Nov 22, 2016",Great Card,While I understand the comments others say abo...


In [157]:
#lets delete any duplicate rows that may have been saved and reset the index
card_review_data = df.drop_duplicates()
card_review_data = card_review_data.reset_index()
card_review_data = card_review_data.drop(['index'],1)

In [162]:
#now lets save that data we just gathered to a csv file
pd.DataFrame(card_review_data).to_csv("creditcardreviews.csv", encoding='utf-8',index = False)